In [2]:
# Import libraries
import pandas as pd

In [3]:
# Load the dataset
df = pd.read_csv("../data/raw/global_findex.csv")

# Display the first few rows
df.head()

C:\Users\Aidid\AppData\Local\Temp\ipykernel_8620\3954426719.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/global_findex.csv")


,countrynewwb,codewb,year,pop_adult,regionwb24_hi,incomegroupwb24,group,group2,account_t_d,fiaccount_t_d,...,con12m_s,con26lm_s,con12w_s,con2f_s,con13_s,con26m_s,con28lm_s,con5a_s,con17c_s,con32h_s
0,Afghanistan,AFG,2011,14575546.0,South Asia (excluding high income),Low income,all,all,0.090050,0.090050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,ALB,2011,2281010.0,Europe & Central Asia (excluding high income),Upper middle income,all,all,0.282681,0.282681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,DZA,2011,26251587.0,Middle East & North Africa (excluding high inc...,Lower middle income,all,all,0.332861,0.332861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Angola,AGO,2011,12779501.0,Sub-Saharan Africa (excluding high income),Lower middle income,all,all,0.392035,0.392035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,ARG,2011,30685516.0,Latin America & Caribbean (excluding high income),Upper middle income,all,all,0.331302,0.331302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Checking the number of columns in the dataset
print(f"Number of columns: {len(df.columns)}")

Number of columns: 437


In [10]:
# Check data type
print(df.dtypes)


countrynewwb      object
codewb            object
year               int64
pop_adult        float64
regionwb24_hi     object
                  ...   
con26m_s         float64
con28lm_s        float64
con5a_s          float64
con17c_s         float64
con32h_s         float64
Length: 437, dtype: object


In [12]:
# Create basic codebook structure
codebook = pd.DataFrame({
    'column_name': df.columns,
    'data_type': [str(df[col].dtype) for col in df.columns],
    'category': '',
    'description': ''
})

# View first 10 rows
codebook.head(10)

,column_name,data_type,category,description
0,countrynewwb,object,,
1,codewb,object,,
2,year,int64,,
3,pop_adult,float64,,
4,regionwb24_hi,object,,
5,incomegroupwb24,object,,
6,group,object,,
7,group2,object,,
8,account_t_d,float64,,
9,fiaccount_t_d,float64,,


In [16]:
# Automatically assign some categories
def assign_category(col_name):
    """
    Assigns each column to a detailed hierarchical category based on its name.
    The categories are ordered by the most unique indicators first (Demographics/Country Info),
    followed by major thematic groups (Connectivity, then Financial), and finally, by other precise groupings.
    """

    # --- 1. Demographics / Country Info ---
    if col_name in ['countrynewwb', 'codewb', 'year', 'regionwb24_hi', 'incomegroupwb24']:
        return 'Demographics / Country Info'
    
    # --- 2.  Demographics /Population Indicators ---
    elif col_name.startswith('pop'):
        return 'Demographics / Population'
    
    # --- 3. Connectivity / Digital
    elif col_name.startswith('con') or col_name in ['dig_acc', 'internet']:
        return 'Connectivity / Digital'
    
    # --- 4. Financial Categories (Thematic Groupings) ---
    elif col_name.startswith('fin') or col_name in ['account_t_d', 'fiaccount_t_d', 'mobileaccount_t_d', 'borrow_any_t_d', 'save_any_t_d', 'merchant_pay']:

        # Financial / Account Ownership & Access
        if any(x in col_name for x in ['account', 'acc', 'fi', 'mobile']):
            return 'Financial / Account Ownership & Access'
        
        # Financial / Borrowing & Credit
        elif any(x in col_name for x in ['borrow', 'bor', 'fin4']):
            return 'Financial / Borrowing & Credit'
        
        # Financial / Payments & Transactions
        elif any(x in col_name for x in ['pay', 'trans', 'merchant', 'send', 'receive']):
            return 'Financial / Payments & Transactions'
        
        # Financial / Savings & Management
        elif any(x in col_name for x in ['save', 'sav', 'g20']):
            return 'Financial / Savings & Management'
        
        # Financial / General Indicators
        else:
            return 'Financial / General Indicators'
        
    # --- 5. Grouping / Internal Index ---
    elif col_name in ['group', 'group2', 'fh1', 'fh2', 'fh2a']:
        return 'Grouping / Internal Index'
    
    # --- 6. Other / Needs Review
    else:
        return 'Other / Needs Review'
    

codebook['category'] = codebook['column_name'].apply(assign_category)

# View First 10 rows
codebook.head(10)

,column_name,data_type,category,description
0,countrynewwb,object,Demographics / Country Info,
1,codewb,object,Demographics / Country Info,
2,year,int64,Demographics / Country Info,
3,pop_adult,float64,Demographics / Population,
4,regionwb24_hi,object,Demographics / Country Info,
5,incomegroupwb24,object,Demographics / Country Info,
6,group,object,Grouping / Internal Index,
7,group2,object,Grouping / Internal Index,
8,account_t_d,float64,Financial / Account Ownership & Access,
9,fiaccount_t_d,float64,Financial / Account Ownership & Access,


In [ ]:
# Save as CSV
codebook.to_csv('data/codebook/Findex_2025_codebook.csv', index=False)